In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
#Used to Load Content from My Drive If you are directly upload content dataset skip this
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/DATASET.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/DATA')



In [4]:
# Paths to dataset folders
TRAIN_DIR = '/content/DATA/DATASET/TRAIN'
TEST_DIR = '/content/DATA/DATASET/TEST'

# Image size and batch size
IMG_SIZE = (128, 128)  # Resize all images to 128x128
BATCH_SIZE = 32


train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(128,128),
    batch_size=32,
    class_mode='binary'   # keep categorical for multiclassification
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'  #keep categorical for multiclassification
)




Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [5]:
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')  # softmax for multi classification
    ])

    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',  #sparse_categorical_crossentropy for multiclassification
                  metrics=['accuracy'])
    return model


model = build_model()
model.summary()

#Training the Model


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)



Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


706/706 ━━━━━━━━━━━━━━━━━━━━ 34s 40ms/step - accuracy: 0.7812 - loss: 0.4938 - val_accuracy: 0.8778 - val_loss: 0.3312
Epoch 2/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 24s 35ms/step - accuracy: 0.8328 - loss: 0.3903 - val_accuracy: 0.8981 - val_loss: 0.2715
Epoch 3/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - accuracy: 0.8548 - loss: 0.3555 - val_accuracy: 0.8926 - val_loss: 0.2767
Epoch 4/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 24s 34ms/step - accuracy: 0.8689 - loss: 0.3219 - val_accuracy: 0.9033 - val_loss: 0.2690
Epoch 5/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 24s 34ms/step - accuracy: 0.8819 - loss: 0.2911 - val_accuracy: 0.8985 - val_loss: 0.2645
Epoch 6/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - accuracy: 0.8914 - loss: 0.2653 - val_accuracy: 0.8894 - val_loss: 0.2851
Epoch 7/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 25s 35ms/step - accuracy: 0.9088 - loss: 0.2284 - val_accuracy: 0.9021 - val_loss: 0.2753
Epoch 8/10
706/706 ━━━━━━━━━━━━━━━━━━━━ 24s 34ms/step - accuracy: 0.9231 - loss: 0.1902 - val_accurac

In [7]:
# 4. Evaluate the Model on Test Data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 3s 34ms/step - accuracy: 0.8902 - loss: 0.4021
Test Accuracy: 0.8857938647270203


In [10]:
# Saving model
model.save('/content/drive/MyDrive/organic_vs_recycle_model.h5')


In [14]:
#Manual Testing of model
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import warnings
warnings.filterwarnings('ignore')

# Load the trained model
model = load_model('/content/drive/MyDrive/organic_vs_recycle_model.h5')

# Path to your custom image
image_path = r"/content/R_4.jpg" # Replace with your image path

# Preprocess the image
IMG_SIZE = (128, 128)  # Must match the size used during training
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(128,128))  # Load and resize image
    img_array = img_to_array(img)  # Convert image to array
    img_array = img_array / 255.0  # Rescale to 0-1 range
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array
#Author: KARAN KATAKDHOND
image = preprocess_image(image_path)

# Make prediction
prediction = model.predict(image)
print("prediction prob" , prediction)
# Interpretation
if prediction[0][0] < 0.5:
    print(f"The image is predicted to be: Organic)")
else:
    print(f"The image is predicted to be: Recyclable")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
prediction prob [[0.9998808]]
The image is predicted to be: Recyclable
